In [1]:
from bs4 import BeautifulSoup as bs
import requests
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)

C:\Users\Chloe\AppData\Local\Temp\ipykernel_10912\3936863406.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"C:\Users\Chloe\Documents\chromedriver_win32\chromedriver.exe",options=options)


In [3]:
# creates a datafram with all of the needed columns including "title", "author", etc.
df = pd.DataFrame({'title': pd.Series(dtype='str'),
                   'director': pd.Series(dtype='str'),
                   'release_year': pd.Series(dtype='str'),
                   'length': pd.Series(dtype='str'),
                   'genre': pd.Series(dtype='str'),
                   'imdb_score': pd.Series(dtype='str'),
                   'rating' : pd.Series(dtype='str'),
                   'gross_US_CA' : pd.Series(dtype='str'),
                   'imdb_votes' : pd.Series(dtype='str')
})

print(df)

#specifies the website to scrape and use selenium
driver.get("https://www.imdb.com/search/title/?title_type=feature&sort=num_votes,desc&after=WzE4NywxNjE1NjYsInR0NjczODA0NiIsODQ2MDFd&ref_=adv_nxt")


i = 0
while True:
    try:
        # waits until the head-block class is displayed
        WebDriverWait(driver, 1).until(
            lambda s: s.find_element(By.CLASS_NAME, 'article').is_displayed()
        )
    except TimeoutException:
        break

    url = driver.current_url
    response = requests.get(url)
    html = response.content
    #creates of beautiful soup object of the current url
    soup = bs(html, "lxml")
    
    #time.sleep(2)
    #makes a list of each div with a class of "item-info"
    all_item_info = soup.find_all("div", class_="lister-item-content")
    #for each item in the list, gather all the relevant data about the books
    for item_info in all_item_info:
        try:
            movie_title = item_info.find("h3", class_="lister-item-header").find("a").get_text(strip=True)
        except:
            movie_title = pd.NA
        try:
            movie_director = item_info.find("p", class_="").find("a").get_text(strip=True)
        except:
            movie_director = pd.NA
        try:
            movie_release = item_info.find("h3", class_="lister-item-header").find("span", class_="lister-item-year text-muted unbold").get_text(strip=True)
        except:
            movie_release = pd.NA
        try:
            movie_length = item_info.find("p", class_="text-muted").find("span", class_="runtime").get_text(strip=True)
        except:
            movie_length = pd.NA
        try:
            movie_genre = item_info.find("p", class_="text-muted").find("span", class_="genre").get_text(strip=True)
        except:
            movie_genre = pd.NA
        try:
            movie_score = item_info.find("div", class_="ratings-bar").find("div", class_="inline-block ratings-imdb-rating").find("strong").get_text(strip=True)
        except:
            movie_score = pd.NA
        try:
            movie_rating = item_info.find("p", class_="text-muted").find("span", class_="certificate").get_text(strip=True)
        except:
            movie_rating = pd.NA
        try:
            movie_gross = item_info.find("p", class_="sort-num_votes-visible").find_all("span")[4].get_text(strip=True)
        except:
            movie_gross = pd.NA
        try:
            movie_votes = item_info.find("p", class_="sort-num_votes-visible").find_all("span")[1].get_text(strip=True)
        except:
            movie_votes = pd.NA
        
        
        #since these values are sometimes non-existent, I use a try-except block
        #to set the value to NaN if there is an error (meaning the value doesn't exist)

        
        #adds each of these values as a new row to the dataframe
        df = pd.concat([df,pd.Series({'title': movie_title,
                       'director': movie_director,
                       'release_year': movie_release,
                       'length': movie_length,
                       'genre' : movie_genre,
                       'imdb_score' : movie_score,
                       'rating' : movie_rating,
                       'gross_US_CA' : movie_gross,
                       'imdb_votes' :movie_votes

                                      
                      }).to_frame().T], ignore_index=True)

    #makes sure the next button exists before trying to click it
    try:
        if (i % 50) == 0:
            df.to_csv('imdb_movies4.csv')
            print(i)
        load_more = driver.find_element(By.LINK_TEXT,'Next »')
        if load_more: driver.execute_script("arguments[0].click();", load_more)
    except: pass
                       

    #after each page it increments i by 1, once i is greater than 33 we have reached
    #the last page so it breaks out of the while statement
    i += 1
    if i > 800:
        break

#saves the dataframe as a csv file
df.to_csv('imdb_movies4.csv')
df

Empty DataFrame
Columns: [title, director, release_year, length, genre, imdb_score, rating, gross_US_CA, imdb_votes]
Index: []
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800


,title,director,release_year,length,genre,imdb_score,rating,gross_US_CA,imdb_votes
0,Aakasha Mittayee,M. Padmakumar,(2017),137 min,Comedy,7.0,<NA>,<NA>,187
1,Eureka Stockade,Harry Watt,(1949),103 min,"Drama, History",6.4,Approved,<NA>,187
2,The Delinquent,Cheh Chang,(1973),100 min,"Drama, Action",6.8,R,<NA>,187
3,Around June,James Savoca,(2008),93 min,"Animation, Drama, Fantasy",5.2,Not Rated,<NA>,187
4,Twisted,Seth Michael Donsky,(1996),100 min,Drama,4.6,Unrated,<NA>,187
...,...,...,...,...,...,...,...,...,...
40045,Royal Shakespeare Company: The Tempest,Gregory Doran,(2017),145 min,Drama,7.8,<NA>,<NA>,68
40046,Looking for a Lady with Fangs and a Moustache,Khyentse Norbu,(2019),113 min,"Drama, Mystery",7.5,<NA>,<NA>,68
40047,Between the Seasons,Junsik Kim,(2018),98 min,Drama,7.2,<NA>,<NA>,68
40048,Un amour interdit,Jean-Pierre Dougnac,(1984),95 min,Drama,5.1,<NA>,<NA>,68
